In [38]:
import xarray as xr
from CMIP6_IO import CMIP6_IO
from dask.distributed import Client
from pathlib import Path
import matplotlib.pyplot as plt

## Create ensemble files from running CMIP6_light.py
This script will loop over all files found in a speficic folder `lightpath`and find files of teh same variable and scenario to create ensemble files from. The various variables to create ensembles for include `["uvb_","uv_","uvi_","par_", "uvb_", "uva_"]`.

The output is stored under folder `ensemble_path` and the results are used to create the final plots of modeled lightlyfor the paper using notebooks `CMIP6_plot_light_results.ipynb`.


In [39]:
# Define what models, scenarios, and variables to calculate ensemble files from
tt = "CMCC-ESM2"

source_ids = ["MPI-ESM1-2-HR", "MPI-ESM1-2-LR", "CanESM5", "UKESM1-0-LL"]

scenarios = ["ssp245"]  # ["ssp245", "ssp585"]
var_names = [
    "par_",
    "uvb_",
    "uv_",
    "uvi_",
    "uva_",
    "uv_srf_",
]  # ,"_tos.nc","_siconc.nc"]
var_names = ["uv_", "uvi_", "uva_"]
var_names = ["uvb_"]  # , "_tos.nc","_siconc.nc"]
var_names = ["par"]  # , "uvb_", "uv_", "uvi_", "uva_"] #, "_tos.nc","_siconc.nc"]

io = CMIP6_IO()

In [40]:
import numpy as np


def collect_ds(scenarios, var_names, ds_list):
    max_models = 300
    for var_name in var_names:
        for scenario in scenarios:
            counter = 0

            if var_name not in ["tos"]:
                lightpath = "light/ncfiles"
            else:
                lightpath = f"light/{scenario}"

            current = f"{lightpath}"
            file_on_gcs = io.list_dataset_on_gs(current)

            # Loop over all files and filter on the models defined in source_ids.
            for f in file_on_gcs:
                # Filter to only use the models we prefer
                if (
                    any(model in f for model in source_ids)
                    and var_name in f
                    and "uv_srf" not in f
                    and scenario in f
                ):
                    if counter >= max_models:
                        pass
                    else:
                        p = Path(f).name
                        model = p.split("_")[1] + "_" + p.split("_")[2]
                        ds = io.open_dataset_on_gs(f)

                        print("=======")
                        print(model)

                        ds_resampled = ds[var_name]
                        ds_list[model] = ds_resampled
                        print("TIME", len(ds_resampled.time.values))
                        print(
                            "PERIOD",
                            ds_resampled.time.values[0],
                            ds_resampled.time.values[-1],
                        )
                        counter += 1

            # Add the ensemble weighted
            if var_name in ["tos"]:
                f = f"{lightpath}/ensemble/CMIP6_ensemble_stats_{scenario}_{var_name}_weighted.nc"
            else:
                f = f"{lightpath}/ensemble/CMIP6_ensemble_stats_{scenario}_{var_name}__weighted.nc"
            ds = io.open_dataset_on_gs(f)

            ds = ds.rename({f"{var_name}_mean": f"{var_name}"})
            ds_list["ensemble_weighted"] = ds[var_name]
            if var_name in ["tos"]:
                f = f"{lightpath}/ensemble/CMIP6_ensemble_stats_{scenario}_{var_name}.nc"
            else:
                f = f"{lightpath}/ensemble/CMIP6_ensemble_stats_{scenario}_{var_name}.nc"
            ds = io.open_dataset_on_gs(f)
            ds = ds.rename({f"{var_name}_mean": f"{var_name}"})
            ds_list["ensemble_unweighted"] = ds[var_name]

    return ds_list


ds_list = {}
ds_list = collect_ds(scenarios, var_names, ds_list)
print(ds_list.keys())
print(f"Number of model-scenario combinations: {len(ds_list.keys())}")

[CMIP6_IO] Opening file actea-shared/light/ncfiles/ssp245/par_CanESM5_r10i1p2f1_1979-01-01-2099-12-16_scenario_osa_ssp245.nc
CanESM5_r10i1p2f1
TIME 8709
PERIOD 1979-01-16T12:00:00.000000000 2099-12-16T20:00:00.000000000
[CMIP6_IO] Opening file actea-shared/light/ncfiles/ssp245/par_CanESM5_r1i1p2f1_1979-01-01-2099-12-16_scenario_osa_ssp245.nc
CanESM5_r1i1p2f1
TIME 8709
PERIOD 1979-01-16T12:00:00.000000000 2099-12-16T20:00:00.000000000
[CMIP6_IO] Opening file actea-shared/light/ncfiles/ssp245/par_CanESM5_r2i1p2f1_1979-01-01-2099-12-16_scenario_osa_ssp245.nc
CanESM5_r2i1p2f1
TIME 8709
PERIOD 1979-01-16T12:00:00.000000000 2099-12-16T20:00:00.000000000
[CMIP6_IO] Opening file actea-shared/light/ncfiles/ssp245/par_CanESM5_r3i1p2f1_1979-01-01-2099-12-16_scenario_osa_ssp245.nc
CanESM5_r3i1p2f1
TIME 8709
PERIOD 1979-01-16T12:00:00.000000000 2099-12-16T20:00:00.000000000
[CMIP6_IO] Opening file actea-shared/light/ncfiles/ssp245/par_CanESM5_r7i1p2f1_1979-01-01-2099-12-16_scenario_osa_ssp245.nc
Ca

In [41]:
min_lon = 5
max_lon = 35
min_lat = 69
max_lat = 79
var_name = "par"
scenario = "ssp245"
if var_name not in ["tos"]:
    lightpath = "light/ncfiles"
else:
    lightpath = f"light/{scenario}"

In [42]:
fig, ax = plt.subplots(figsize=(16, 8))

saved_model_name = None
color_index = 0
for model in ds_list.keys():
    ds = ds_list[model]
    if var_name in ["tos"]:
        ds = ds.sel(y=slice(min_lat, max_lat), x=slice(min_lon, max_lon)).mean(
            {"y", "x"}
        )
    else:
        ds = ds.sel(lat=slice(min_lat, max_lat), lon=slice(min_lon, max_lon)).mean(
            {"lat", "lon"}
        )
    print(ds.time)
    print(ds.time.values[0:4])

    ds = ds.resample(time="A").mean(skipna=True)
    print(f"Number of points: {ds.count(...)}")
    colors = plt.cm.tab20((4.0 / 3 * np.arange(20 * 3 / 4)).astype(int))
    model_name = model.split("_")[0]

    if saved_model_name is None or saved_model_name != model_name:
        saved_model_name = model_name
        color = colors[color_index]
        color_index += 1
    linewidth = 0.5
    alpha = 0.8
    print(f"Running calculations and plotting {model}")

    if model in ["ensemble_weighted"]:
        color = "tab:orange"
        linewidth = 2.0
        alpha = 1.0
    if model in ["ensemble_unweighted"]:
        color = "tab:blue"
        linewidth = 2.0
        alpha = 1.0

    ax.plot(
        ds.time.values,
        ds,
        linewidth=linewidth,
        color=color,
        alpha=alpha,
        label=f"{model}",
    )

ax.legend(
    title="Model-scenario",
    fontsize=8,
    title_fontsize=10,
    loc="upper left",
    ncol=2,
    handleheight=2.4,
    labelspacing=0.05,
)

<xarray.DataArray 'time' (time: 8709)>
array(['1979-01-16T12:00:00.000000000', '1979-01-16T16:00:00.000000000',
       '1979-01-16T20:00:00.000000000', ..., '2099-12-16T12:00:00.000000000',
       '2099-12-16T16:00:00.000000000', '2099-12-16T20:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 1979-01-16T12:00:00 ... 2099-12-16T20:00:00
Attributes:
    long_name:  days since 2000-01-16 00:00:00
    field:      time, scalar, series
['1979-01-16T12:00:00.000000000' '1979-01-16T16:00:00.000000000'
 '1979-01-16T20:00:00.000000000' '1979-02-16T00:00:00.000000000']
Number of points: 121
Running calculations and plotting CanESM5_r10i1p2f1
<xarray.DataArray 'time' (time: 8709)>
array(['1979-01-16T12:00:00.000000000', '1979-01-16T16:00:00.000000000',
       '1979-01-16T20:00:00.000000000', ..., '2099-12-16T12:00:00.000000000',
       '2099-12-16T16:00:00.000000000', '2099-12-16T20:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * ti